# Ch3: Traditional Symmetric Key Ciphers
## 3. 1 Monoalphabetic Ciphers
### 3.1.1 Caesar Shift Cipher (Additive Cipher)
In the Caeser shift cipher the plaintext character $x$ is transformed into `(x - 'a' + m) % 26 + 'a'`

In [1]:
def caeser_shift(plaintext, shift=0):
    return ''.join([chr((ord(letter) - ord('a') + shift) % 26 + 97) for letter in plaintext])

In [2]:
ciphertext = caeser_shift('hello', 15)
print(ciphertext)

wtaad


In [3]:
print(caeser_shift(ciphertext, -15))

hello


In [8]:
from collections import Counter
def get_frequencies(ciphertext):
    return Counter(ciphertext)

### 3.1.2 Performing Statistical Analysis on Additive Ciphers

In [9]:
print(get_frequencies(ciphertext))

Counter({'a': 2, 'w': 1, 't': 1, 'd': 1})


### 3.1.3 Multiplicative Cipher
The Ciphertext $C = (P * k) \% 26$ where k is the key and for decrytping it is $P = (C * k^{-1}) \% 26$ 

In [4]:
def multiplicative_cipher_encrypt(plaintext, key):
    return ''.join([chr(((ord(letter) - ord('a')) * key) % 26 + ord('a')) for letter in plaintext.lower()])

In [5]:
def gcd(a, b):
    return gcd(b, a % b) if b != 0 else a


def extended_gcd(a, b):
    s, old_s = 1, 0
    t, old_t = 0, 1

    while b != 0:
        q = a // b
        a, b = b, a % b
        s, old_s = old_s, s - q * old_s
        t, old_t = old_t, t - q * old_t
    return a, s, t


def multiplicative_inverse_exists(a, n):
    return gcd(a, n) == 1


def multiplicative_inverse(b, n):
    if multiplicative_inverse_exists(b, n):
        return (extended_gcd(n, b)[2] + n) % n
    

def multiplicative_cipher_decrypt(ciphertext, key):
    return ''.join(
        [chr(((ord(letter) - ord('a')) * multiplicative_inverse(key, 26)) % 26 + ord('a')) for letter in ciphertext.lower()]
    )

In [6]:
print(multiplicative_cipher_encrypt('hello', 7))

xczzu


In [13]:
print(multiplicative_cipher_decrypt('xczzu', 7))

hello


### 3.1.3 Affine Ciphers
The affine cipher takes in 2 keys $k_1$ and $k_2$ and encrypte the plaintext code $P$ as follows $C = (P * k_1 + k_2) \% 26$. It decrpyts the ciphertext as follows: $P = ((C - k_2) * k_1^{-1}) \% 26$. Let us create a class AffineCipher that will offer teh user with 2 methods to encrypt and decrpyt data.

In [14]:
class AffineCipher:
    def __init__(self, k_1, k_2):
        self.k_1 = k_1
        self.k_2 = k_2
        
    def char_to_num(self, letter):
        letter = letter.lower()
        return ord(letter) - ord('a')
        
    def encrypt(self, plaintext):
        return ''.join([chr((self.char_to_num(letter) * self.k_1 + self.k_2) % 26 + ord('A')) for letter in plaintext.lower()])
    
    def decrypt(self, ciphertext):
        return ''.join(
            [chr(((self.char_to_num(letter) - self.k_2) * multiplicative_inverse(self.k_1, 26)) % 26 + ord('a'))
             for letter in ciphertext.lower()]
        )

In [15]:
cipher = AffineCipher(11, 3)
cipher.encrypt('hello')

'CVUUB'

In [16]:
cipher.decrypt('CVUUB')

'hello'

## 3.2 Polyalphabetic Ciphers
### 3.2.1 Autokey Cipher
In the autokey cipher the initial key $k$ is given and all subsequent keys are generated using the plaintext characters hence the keys become $(k, P_1, P_2, P_3 \cdots)$.  

In [17]:
class AutoKeyCipher:
    def __init__(self, k):
        self.k = k
        
    def encrypt(self, plaintext):
        key = self.k
        ciphertext = ''
        for letter in plaintext.lower():
            ciphertext += chr((ord(letter) - ord('a') + key) % 26 + ord('A'))
            key = ord(letter) - ord('a')
        return ciphertext
    
    def decrypt(self, ciphertext):
        key = self.k
        plaintext = ''
        for letter in ciphertext.lower():
            key = (ord(letter) - ord('a') - key) % 26
            plaintext += chr(key + ord('a'))
        return plaintext

In [18]:
auto_key_cipher = AutoKeyCipher(12)
auto_key_cipher.encrypt('attackistoday')

'MTMTCMSALHRDY'

In [19]:
auto_key_cipher.decrypt('MTMTCMSALHRDY')

'attackistoday'

### 3.2.2 Playfair Cipher
The Playfair Cipher takes in an initial 5x5 matrix where __i__ and __j__ share the same spot and the encryption follows the following rules:

$$
\begin{bmatrix}
L & G & D & B & A \\
Q & M & H & E & C \\
U & R & N & I/J & F \\
X & V & S & O & K \\
Z & Y & W & T & P \\
\end{bmatrix}
$$

1. Add an extra character when consecutive letters are the same.
1. Pad an extra character at the end if the length of the plaintext is odd.
1. Take pairs at a time and
    1. If the pairs belong to the same row then the enncrypted character is in the same row just shifted +1 to the right (with wrapping in the same row).
    1. If the pairs belong to the same column then the enncrypted character is in the same column just shifted +1 to the bottom (with wrapping in the same column).
    1. If the pairs are in different rows and columns then take character from first letter's row and second letter's column and then vica versa.

In [7]:
class PlayFairCipher:
    def __init__(self, key):
        self.key = key
        self.letter2index = self.get_letter_2_index_map()

    def get_letter_2_index_map(self):
        index_map = {}
        for row in range(len(self.key)):
            for column in range(len(self.key[row])):
                letter = self.key[row][column]
                if letter == 'i':
                    index_map['j'] = (row, column)
                index_map[letter] = (row, column)
        return index_map

    def encrypt(self, plaintext):
        plaintext = self.pad_extra(self.remove_consecutive_same_chars(plaintext.lower()))
        # print('plaintext:', plaintext)
        ciphertext = ''
        for index in range(0, len(plaintext), 2):
            pair = plaintext[index: index + 2]
            ciphertext += self.encipher(pair)
        return ciphertext.upper()

    def decrypt(self, ciphertext):
        plaintext = ''
        for index in range(0, len(ciphertext), 2):
            pair = ciphertext[index: index + 2].lower()
            plaintext += self.decipher(pair)
        return plaintext

    def decipher(self, pair):
        row1, column1 = self.letter2index[pair[0]]
        row2, column2 = self.letter2index[pair[1]]
        if row1 == row2:
            return self.key[row1][(column1 - 1) % 5] + self.key[row1][(column2 - 1) % 5]
        elif column1 == column2:
            return self.key[(row1 - 1) % 5][column1] + self.key[(row2 - 1) % 5][column2]
        return self.key[row1][column2] + self.key[row2][column1]

    def encipher(self, pair):
        row1, column1 = self.letter2index[pair[0]]
        row2, column2 = self.letter2index[pair[1]]
        if row1 == row2:
            return self.key[row1][(column1 + 1) % 5] + self.key[row1][(column2 + 1) % 5]
        elif column1 == column2:
            return self.key[(row1 + 1) % 5][column1] + self.key[(row2 + 1) % 5][column2]
        return self.key[row1][column2] + self.key[row2][column1]

    @staticmethod
    def pad_extra(plaintext):
        return plaintext + chr((ord(plaintext[len(plaintext) - 1]) - ord('a') + 1) % 26 +ord('a')) \
            if len(plaintext) % 2 == 1 else plaintext

    @staticmethod
    def remove_consecutive_same_chars(plaintext):
        for index in range(len(plaintext) - 1):
            if plaintext[index] == plaintext[index + 1]:
                plaintext = plaintext[: index + 1] + chr((ord(plaintext[index]) - ord('a') + 1) % 26 + ord('a')) \
                            + plaintext[index + 1:]
        return plaintext

In [10]:
playfair_cipher = PlayFairCipher(
    [['l', 'g', 'd', 'b', 'a'],
     ['q', 'm', 'h', 'e', 'c'],
     ['u', 'r', 'n', 'i', 'f'],
     ['x', 'v', 's', 'o', 'k'],
     ['z', 'y', 'w', 't', 'p']]
)

ciphertext = playfair_cipher.encrypt('iwillattacktomorrow')
print(ciphertext)

NTUBQGBPIZCFOPVEVIVNST


In [11]:
plaintext = playfair_cipher.decrypt('NTUBQGBPIZCFOPVEVIVNST')
print(plaintext)

iwilmlatutacktomorsrow


### 3.2.3 Vignere Cipher
The vignere cipher takes in a string as a key and uses individual letters of the keys at different positions (indices) to encrypt the data. $C = (P_i + K_i) \% 26$ and $P = (C_i - K_i) \% 26$ 

In [12]:
class VignereCipher:
    def __init__(self, key: str):
        self.key = key

    def encrypt(self, plaintext: str) -> str:
        ciphertext = ''
        for index, letter in enumerate(plaintext.lower()):
            ciphertext += chr((self.char_2_num(letter) + self.char_2_num(self.key[index % len(self.key)])) % 26 + ord('a'))
        return ciphertext.upper()

    def decrypt(self, ciphertext: str) -> str:
        plaintext = ''
        for index, letter in enumerate(ciphertext.lower()):
            plaintext += chr(
                (self.char_2_num(letter) - self.char_2_num(self.key[index % len(self.key)])) % 26 + ord('a'))
        return plaintext.lower()

    @staticmethod
    def char_2_num(character: str) -> int:
        return ord(character.lower()) - ord('a')

In [13]:
vignere_cipher = VignereCipher('pascal')
ciphertext = vignere_cipher.encrypt('sheislistening')
print(ciphertext)

HHWKSWXSLGNTCG


In [14]:
print(vignere_cipher.decrypt(ciphertext))

sheislistening


### 3.2.4 Hill Cipher
In the Hill Cipher we are given a matrix of size __m X m__ which is our key in this instance and we divide the plaintext we get into __l__ blocks of size __m__. So we get a __l x m__ matric from the input plaintext. The Ciphertext __C__ from Plaintext __P__ can be extracted as $C = (P * K) % 26$ and similarly can be decrypted as $P = (C * K^{-1}) %26$.

> Note: The Key $K$ must have a modular inverse in $Z_{26}$ otherswise the message won't be decryptable. 

In [26]:
import numpy as np
from numpy import linalg

# Defining a few helper utility functions
def modMatInv(A, p):  # Finds the inverse of matrix A mod p
    n = len(A)
    adj = np.zeros(shape=(n, n))
    for i in range(0, n):
        for j in range(0, n):
            adj[i][j] = ((-1) ** (i + j) * int(round(linalg.det(minor(A, j, i))))) % p
    return (modInv(int(round(linalg.det(A))), p) * adj) % p

def modInv(a, p):  # Finds the inverse of a mod p, if it exists
    for i in range(1, p):
        if (i * a) % p == 1:
            return i
    raise ValueError(str(a) + " has no inverse mod " + str(p))


def minor(A, i, j):  # Return matrix A with the ith row and jth column deleted
    A = np.array(A)
    sub_matrix = np.zeros(shape=(len(A) - 1, len(A) - 1))
    p = 0
    for s in range(0, len(sub_matrix)):
        if p == i:
            p = p + 1
        q = 0
        for t in range(0, len(sub_matrix)):
            if q == j:
                q = q + 1
            sub_matrix[s][t] = A[p][q]
            q = q + 1
        p = p + 1
    return sub_matrix

In [27]:
import numpy as np


class HillCipher:
    def __init__(self, key):
        self.key = key
        self.block_size = len(key)
        self.key_inv = modMatInv(self.key, 26)

    def encrypt(self, plaintext: str) -> str:
        P = self.text_2_mat(plaintext)
        C = P.dot(self.key) % 26
        return self.mat_2_text(C).upper()

    def decrypt(self, ciphertext: str) -> str:
        C = self.text_2_mat(ciphertext)
        P = C.dot(self.key_inv) % 26
        return self.mat_2_text(P).lower()

    def text_2_mat(self, text: str):
        text = self.pad_chars(text.lower()).lower()
        l = len(text) // self.block_size
        return np.reshape(list(map(self.char_2_num, text)), (l, self.block_size))

    def mat_2_text(self, matrix) -> str:
        return ''.join(list(map(self.num_2_char, list(matrix.ravel()))))

    @staticmethod
    def char_2_num(character: str) -> int:
        character = character.lower()
        return ord(character) - ord('a')

    @staticmethod
    def num_2_char(number: int) -> str:
        return chr(int(number) + ord('A'))

    def pad_chars(self, plaintext: str) -> str:
        padding = (- (len(plaintext) % self.block_size)) % self.block_size
        return plaintext + 'z' * padding

In [28]:
hill_cipher = HillCipher([
    [9, 7, 11, 13],
    [4, 7, 5, 6],
    [2, 21, 14, 9],
    [3, 23, 21, 8]
])

ciphertext = hill_cipher.encrypt('codeisready')
print(ciphertext)

OHKNIHGHFISS


In [29]:
print(hill_cipher.decrypt(ciphertext))

codeisreadyz


### 3.2.5 One Time Pad
In the one time pad cipher a random key will be generated everytime of the same length as the plaintext message and only this one message can be encrypted with it and then decrypted using the one time pad. After this message has been exncrypted and created. No new messages can be created from that pad and the pad needs to be discarded and a new pad needs to be used.

In [30]:
import numpy as np


class OneTimePadCipher:
    def __init__(self):
        self.key = ''
        self.pad_used = False
        self.cipher = None

    def encrypt(self, plaintext: str):
        if not self.pad_used:
            self.pad_used = True
            self.generate_random_key(plaintext)
            self.cipher = VignereCipher(self.key)
            return self.cipher.encrypt(plaintext)
        else:
            return None

    def decrypt(self, ciphertext: str):
        if not self.pad_used:
            return None
        else:
            return self.cipher.decrypt(ciphertext)

    def generate_random_key(self, plaintext: str) -> None:
        self.key = ''.join(list(map(self.num_2_char, np.random.randint(0, 25, len(plaintext)))))

    @staticmethod
    def num_2_char(number: int) -> str:
        return chr(number + ord('a'))

In [31]:
# using a one time pad to encrypt data
cipher = OneTimePadCipher()
ciphertext = cipher.encrypt('helloworld')
print(ciphertext)

KBCSSFLYJJ


In [33]:
# can't use this pad to encrypt anything ever again
print(cipher.encrypt('newdata'))

None


In [34]:
# we can try to decrypt as many strings as we want using this pad, but only our correct ciphertext wil give the desired output
print(cipher.decrypt('thisisatest'))

qkrlejdmgmq


In [35]:
print(cipher.decrypt('KBCSSFLYJJ'))

helloworld


## 3.3 Transposition Ciphers
### 3.3.1 KeyLess Transposition Ciphers
#### 3.3.1.1 Column Transposition Cipher
In the column transposition cipher the plain text is written column wise over $r$ rows and then the ciphertext is simply the text taken over the rows. 

In [36]:
import numpy as np
from math import ceil


class ColumnTranspositionCipher:
    def __init__(self, rows):
        self.rows = rows

    def encrypt(self, plaintext: str) -> str:
        ciphertext = ''
        for i in range(self.rows):
            for j in range(i, len(plaintext), self.rows):
                ciphertext += plaintext[j]
        return ciphertext.upper()

    def decrypt(self, ciphertext: str) -> str:
        columns = ceil(len(ciphertext) / self.rows)
        padding = len(ciphertext) % self.rows
        ciphertext += 'z' * padding
        C = np.reshape(list(ciphertext), (self.rows, columns))
        return ''.join(C.T.ravel())[: len(ciphertext) - padding].lower()

In [37]:
column_transpose_cipher = ColumnTranspositionCipher(2)
ciphertext = column_transpose_cipher.encrypt('meetmeatthepark')
print(ciphertext)

MEMATEAKETETHPR


In [38]:
print(column_transpose_cipher.decrypt('MEMATEAKETETHPR'))

meetmeatthepark


#### 3.3.1.2 Rail Fence Transposition Cipher
This is a special case of the column Transposition Cipher and here the number of rows is predefined as 2.

In [39]:
class RailFenceCipher:
    def __init__(self):
        self.cipher = ColumnTranspositionCipher(rows=2)

    def encrypt(self, plaintext: str) -> str:
        return self.cipher.encrypt(plaintext)

    def decrypt(self, ciphertext: str) -> str:
        return self.cipher.decrypt(ciphertext)

In [42]:
rail_fence_cipher = RailFenceCipher()
ciphertext = rail_fence_cipher.encrypt('meetmeatthepark')
print(ciphertext)

MEMATEAKETETHPR


In [43]:
print(rail_fence_cipher.decrypt('MEMATEAKETETHPR'))

meetmeatthepark


#### 3.3.1.3 Row Transposition Cipher
The Row transposition takes in a number of columns and wraps the text around columsn and returns rows. This is inherently equivalent to the Column Transposition Cipher.

In [45]:
class RowTranspositionCipher:
    def __init__(self, columns):
        self.cipher = ColumnTranspositionCipher(rows=columns)

    def encrypt(self, plaintext: str) -> str:
        return self.cipher.encrypt(plaintext)

    def decrypt(self, ciphertext: str) -> str:
        return self.cipher.decrypt(ciphertext)

### 3.3.2 Key Based Transposition Cipher
#### 3.3.2.1 Keyed Transposition Cipher
Takes in a key that gives the order for a block to be re-arranged. Divides the original message with padding and rearranges each block with permutation based key.

$$
\begin{pmatrix}
3 & 1 & 4 & 5 & 2 \\
1 & 2 & 3 & 4 & 5
\end{pmatrix}
$$

In [49]:
class KeyedTranspositionCipher:
    def __init__(self, key):
        self.key = key
        self.decryption_permutation = self.get_decryption_permutation()

    def encrypt(self, plaintext: str) -> str:
        padding = (- (len(plaintext) % len(self.key))) % len(self.key)
        plaintext = plaintext.lower() + 'z' * padding
        ciphertext = ''
        for index in range(0, len(plaintext), len(self.key)):
            block = plaintext[index: index + len(self.key)]
            ciphertext += self.encipher(block)
        return ciphertext.upper()

    def decrypt(self, ciphertext: str) -> str:
        plaintext = ''
        for index in range(0, len(ciphertext), len(self.key)):
            block = ciphertext[index: index + len(self.key)].lower()
            plaintext += self.decipher(block)
        return plaintext

    def decipher(self, block: str) -> str:
        return self.crypt(block, key=self.decryption_permutation)

    def encipher(self, block: str) -> str:
        return self.crypt(block, key=self.key)

    def crypt(self, block: str, key):
        encrypted = ['a'] * len(self.key)
        for index, letter in enumerate(block):
            encrypted[key[index]] = letter
        return ''.join(encrypted)

    def get_decryption_permutation(self):
        permutation = [0] * len(self.key)
        for index, value in enumerate(self.key):
            permutation[value] = index
        return permutation

In [50]:
key_transposition_cipher = KeyedTranspositionCipher(key=[1, 4, 0, 2, 3])
ciphertext = key_transposition_cipher.encrypt('enemyattackstonight')
print(ciphertext)

EEMYNTAACTTKONSHITZG


In [51]:
print(key_transposition_cipher.decrypt(ciphertext))

enemyattackstonightz


#### 3.3.2.2 Columnar Transposition Cipher
We encrypt a given Plaintext $P$ using first a keyless transposition cipher, then key based permutation cipher then row based unrolling. We get initial plaintext __enemy attacks tonight__. We also have permutation key $K$ as:

$$
\begin{pmatrix}
3 & 1 & 4 & 5 & 2 \\
1 & 2 & 3 & 4 & 5
\end{pmatrix}
$$

We first wrap around the plaintext in a matrix with with $c$ columns where $c = |K|$

$$
\begin{bmatrix}
e & n & e & m & y \\
a & t & t & a & c \\
k & s & t & o & n \\
i & g & h & t & z
\end{bmatrix}
$$

We then use the key $K$ to permutate the columns

$$
\begin{bmatrix}
E & E & M & Y & N \\
T & A & A & C & T \\
T & K & O & N & S \\
H & I & T & Z & G 
\end{bmatrix}
$$

We then create the ciphertext by reading columns 1-by-one and a simlar pipeline is followed in reverse to obtain plaintext from the ciphertext.

$C = ETTHEAKIMAOTYCNZNTSG$

In [52]:
import numpy as np

class ColumnarTranspositionCipher:
    def __init__(self, key: list):
        self.key = key
        self.key_inv = self.inv(key)

    def encrypt(self, plaintext: str) -> str:
        A = self.str_2_mat(plaintext)
        A = A[:, self.key]
        return self.mat_2_str(A.T).upper()

    def decrypt(self, ciphertext: str) -> str:
        C = self.str_2_mat_column_wise(ciphertext)
        C = C[:, self.key_inv]
        return self.mat_2_str(C).lower()

    def str_2_mat_column_wise(self, text: str):
        rows, columns = len(self.key), len(text) // len(self.key)
        return np.reshape(list(text), (rows, columns)).T

    def str_2_mat(self, text: str):
        padding = (- (len(text) % len(self.key))) % len(self.key)
        text = text + 'z' * padding
        rows, columns = len(text) // len(self.key), len(self.key)
        return np.reshape(list(text), (rows, columns))

    @staticmethod
    def inv(key: list) -> list:
        inv_key = [0] * len(key)
        for index, value in enumerate(key):
            inv_key[value] = index
        return inv_key

    @staticmethod
    def mat_2_str(matrix) -> str:
        return ''.join(matrix.ravel())

In [53]:
key_transposition_cipher = ColumnarTranspositionCipher(key=[2, 0, 3, 4, 1])
ciphertext = key_transposition_cipher.encrypt('enemyattackstonight')
print(ciphertext)

ETTHEAKIMAOTYCNZNTSG


In [54]:
print(key_transposition_cipher.decrypt(ciphertext))

enemyattackstonightz
